<a href="https://colab.research.google.com/github/mtrefilek/cs762/blob/main/Feature_Extractor_(CLIP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import CLIPTokenizer, CLIPProcessor, CLIPModel
import torch

In [ ]:
import os
cwd = os.getcwd().replace('\\','/')

In [ ]:
#%%capture
PRETRAINED_MODEL_NAME = 'clip-vit-base-patch32'

### Model Preparation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = cwd+'/pretrained_models/'+PRETRAINED_MODEL_NAME

# model = CLIPModel.from_pretrained('openai/'+PRETRAINED_MODEL_NAME)
# processor = CLIPProcessor.from_pretrained('openai/'+PRETRAINED_MODEL_NAME)
# model.save_pretrained(model_path)
# processor.save_pretrained(model_path)

model = CLIPModel.from_pretrained(model_path)
processor = CLIPProcessor.from_pretrained(model_path)
model.eval()
model.to(device)

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0): CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): CLIPEncode

In [ ]:
import numpy as np
import pickle
from copy import deepcopy

In [ ]:
DSET_NAME = 'EuroSAT' #('MNIST', 'FMNIST', 'CIFAR10', 'CIFAR100', 'PlantDisease', 'EuroSAT', 'ChestXRay') 'ISIC2018', 'TinyImageNet' are not yet implemented

In [ ]:
### Dataset Preparation
from torchvision import datasets
from glob import glob, iglob
from PIL import Image

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

#import torchvision
#transform=torchvision.transforms.Compose([
    # you can add other transformations in this list
    #torchvision.transforms.ToTensor()
#])
dset_path = cwd + '/dataset'
feature_path = cwd+'/extracted_features/'

if DSET_NAME=='MNIST': # MNIST
    mnist_train = datasets.MNIST(root=dset_path, train=True, download=True)#, transform=transform)
    mnist_test = datasets.MNIST(root=dset_path, train=False, download=True)#, transform=transform)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in mnist_train:
        imgs_tr.append(img.convert(mode='RGB'))
        labels_tr.append(label)
    for (img, label) in mnist_test:
        imgs_tst.append(img.convert(mode='RGB'))
        labels_tst.append(label)
    classnames = [str(a) for a in range(10)]
    t = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
    classtexts = ['This is number '+a+', '+t[int(a)]+'.' for a in classnames]

elif DSET_NAME=='FMNIST': # Fashion-MNIST
    fmnist_train = datasets.FashionMNIST(root=dset_path, train=True, download=True)#, transform=transform)
    fmnist_test = datasets.FashionMNIST(root=dset_path, train=False, download=True)#, transform=transform)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in fmnist_train:
        imgs_tr.append(img.convert(mode='RGB'))
        labels_tr.append(label)
    for (img, label) in fmnist_test:
        imgs_tst.append(img.convert(mode='RGB'))
        labels_tst.append(label)
    classnames = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    classtexts = []
    for a in classnames:
        if a[0] in 'aeiouAEIOU':
            classtexts.append('This is a picture of an '+a+'.')
        else:
            classtexts.append('This is a picture of a '+a+'.')

elif DSET_NAME=='CIFAR10': # CIFAR-10
    cifar10_train = datasets.CIFAR10(root=dset_path, train=True, download=True)#, transform=transform)
    cifar10_test = datasets.CIFAR10(root=dset_path, train=False, download=True)#, transform=transform)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in cifar10_train:
        imgs_tr.append(img)
        labels_tr.append(label)
    for (img, label) in cifar10_test:
        imgs_tst.append(img)
        labels_tst.append(label)
    meta = pickle.load( open(dset_path+'/cifar-10-batches-py/batches.meta','rb') )
    classnames = meta['label_names']
    classtexts = []
    for a in classnames:
        if a[0] in 'aeiouAEIOU':
            classtexts.append('This is a picture of an '+a+'.')
        else:
            classtexts.append('This is a picture of a '+a+'.')

elif DSET_NAME=='CIFAR100': # CIFAR-100
    cifar100_train = datasets.CIFAR100(root=dset_path, train=True, download=True)#, transform=transform)
    cifar100_test = datasets.CIFAR100(root=dset_path, train=False, download=True)#, transform=transform)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in cifar100_train:
        imgs_tr.append(img)
        labels_tr.append(label)
    for (img, label) in cifar100_test:
        imgs_tst.append(img)
        labels_tst.append(label)
    meta = pickle.load( open(dset_path+'/cifar-100-python/meta','rb') )
    classnames = meta['fine_label_names']
    classtexts = []
    for a in classnames:
        if a[0] in 'aeiouAEIOU':
            classtexts.append('This is a picture of an '+a+'.')
        else:
            classtexts.append('This is a picture of a '+a+'.')

elif DSET_NAME in ('PlantDisease','EuroSAT'): # PlantDisease or EuroSAT
    list_dirs = glob(dset_path+'/'+DSET_NAME+'/*/')
    list_dirs = [l.replace('\\','/') for l in list_dirs]
    classnames = [d.split('/')[-2] for d in list_dirs]
    classtexts = []
    if DSET_NAME == 'PlantDisease':
        for c in classnames:
            a = c.split('___')[0].replace('_', ' ')
            b = c.split('___')[1].replace('_', ' ')
            if b == 'healthy':
                ab = b + ' ' + a
            else:
                ab = a + ' with ' + b
            if ab[0] in 'aeiouAEIOU':
                ab = 'an ' + ab
            else:
                ab = 'a ' + ab
            classtexts.append('This is a picture of '+ab+'.')
    elif DSET_NAME == 'EuroSAT':
        for a in classnames:
            if a in ('Industrial', 'Residential'):
                a += ' area'
            if a[0] in 'aeiouAEIOU':
                classtexts.append('This is a satellite picture of an '+a+'.')
            else:
                classtexts.append('This is a satellite picture of a '+a+'.')
    feature_matrices = []
    labels = []
    for i, classname in enumerate(classnames):
        imgs = []
        for f in iglob(dset_path+'/'+DSET_NAME+'/'+classname+'/*'):
            tmp = Image.open(f)
            img = tmp.copy().convert(mode='RGB')
            imgs.append(img)
        print(classname)
        feature_matrix = np.zeros((1, 512))
        for img_batch in batch(imgs, n=128):
            inputs = processor(images=img_batch, return_tensors="pt")
            img_features = model.get_image_features(inputs['pixel_values'].to(device)).cpu().detach().numpy()
            feature_matrix = np.concatenate((feature_matrix, img_features), axis=0)
        feature_matrices.append(deepcopy(feature_matrix[1:]))
        print(classname)
    inputs = processor(text=classtexts, return_tensors="pt", padding=True).to(device)
    classtext_embeddings = model.get_text_features(**inputs).cpu().detach().numpy()
    fname = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'.npz'
    np.savez(feature_path + fname, feature_matrices = feature_matrices, classnames = classnames, classtexts=classtexts, classtext_embeddings = classtext_embeddings)

elif DSET_NAME=='ChestXRay': # Chest X-Ray
    for split in ('train', 'test'):
        list_dirs = glob(dset_path+'/'+DSET_NAME+'/'+split+'/*/')
        list_dirs = [l.replace('\\','/') for l in list_dirs]
        classnames = [d.split('/')[-2] for d in list_dirs]
        classtexts = []
        for a in classnames:
            if a=='NORMAL':
                classtexts.append('This is a chest x-ray image of a '+a.lower()+' (healty) person.')
            elif a=='PNEUMONIA':
                classtexts.append('This is a chest x-ray image of a patient with '+a.lower()+'.')
        feature_matrices = []
        labels = []
        for i, classname in enumerate(classnames):
            feature_matrix = np.zeros((1, 512))
            for f in iglob(dset_path+'/'+DSET_NAME+'/'+split+'/'+classname+'/*'):
                tmp = Image.open(f)
                img = tmp.copy().convert(mode='RGB')
                inputs = processor(images=img, return_tensors="pt")
                img_features = model.get_image_features(inputs['pixel_values'].to(device)).cpu().detach().numpy()
                feature_matrix = np.concatenate((feature_matrix, img_features), axis=0)
            feature_matrices.append(deepcopy(feature_matrix[1:]))
            print(classname)
        inputs = processor(text=classtexts, return_tensors="pt", padding=True).to(device)
        classtext_embeddings = model.get_text_features(**inputs).cpu().detach().numpy()
        fname = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'_'+split+'.npz'
        np.savez(feature_path + fname, feature_matrices = feature_matrices, classnames = classnames, classtexts=classtexts, classtext_embeddings = classtext_embeddings)

### Extract Features
if DSET_NAME in ('MNIST', 'FMNIST', 'CIFAR10', 'CIFAR100'):
    n_tr = len(imgs_tr)
    feature_matrix_tr = np.zeros((1, 512))
    for img_batch in batch(imgs_tr, n=128):
        inputs = processor(images=img_batch, return_tensors="pt")
        img_features_tr = model.get_image_features(inputs['pixel_values'].to(device)).cpu().detach().numpy()
        feature_matrix_tr = np.concatenate((feature_matrix_tr, img_features_tr), axis=0)
        print('Extracting Training Features: {0:.2f}% done'.format(100*len(feature_matrix_tr[1:])/n_tr) )
    feature_matrix_tr = feature_matrix_tr[1:]

    n_cls = np.max(labels_tr)+1
    labels_tr = np.array(labels_tr)
    feature_matrices_tr = []
    for i in range(n_cls):
        feature_matrices_tr.append(feature_matrix_tr[labels_tr==i])

    n_tst = len(imgs_tst)
    feature_matrix_tst = np.zeros((1, 512))
    for img_batch in batch(imgs_tst, n=128):
        inputs = processor(images=img_batch, return_tensors="pt")
        img_features_tst = model.get_image_features(inputs['pixel_values'].to(device)).cpu().detach().numpy()
        feature_matrix_tst = np.concatenate((feature_matrix_tst, img_features_tst), axis=0)
        print('Extracting Test Features: {0:.2f}% done'.format(100*len(feature_matrix_tst[1:])/n_tst) )
    feature_matrix_tst = feature_matrix_tst[1:]

    n_cls = np.max(labels_tst)+1
    labels_tst = np.array(labels_tst)
    feature_matrices_tst = []
    for i in range(n_cls):
        feature_matrices_tst.append(feature_matrix_tst[labels_tst==i])
        
    inputs = processor(text=classtexts, return_tensors="pt", padding=True).to(device)
    classtext_embeddings = model.get_text_features(**inputs).cpu().detach().numpy()
    
### Save Features
if DSET_NAME in ('MNIST', 'FMNIST', 'CIFAR10', 'CIFAR100'):
    fname_tr = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'_train.npz'
    fname_tst = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'_test.npz'

    np.savez(feature_path + fname_tr, feature_matrices = feature_matrices_tr, classnames = classnames, classtexts = classtexts, classtext_embeddings = classtext_embeddings)
    np.savez(feature_path + fname_tst, feature_matrices = feature_matrices_tst, classnames = classnames, classtexts = classtexts, classtext_embeddings = classtext_embeddings)

Annual Crop


C:\Users\user\anaconda3\lib\site-packages\transformers\feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  tensor = as_tensor(value)


Annual Crop
Forest
Forest
Herbaceous Vegetation
Herbaceous Vegetation
Highway
Highway
Industrial
Industrial
Pasture
Pasture
Permanent Crop
Permanent Crop
Residential
Residential
River
River
Sea Lake
Sea Lake


C:\Users\user\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
